In [ ]:
import pandas as pd
import numpy as np

In [ ]:
MVCollisions = pd.read_csv('Motor_Vehicle_Collisions_-_Crashes_20251118.csv')
MVCollisions.head()

In [ ]:
MVCollisions.columns

In [ ]:
!pip install google-cloud-storage

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/patkaa/Documents/Assignment 1 - DW - Fall 2025/mvcollisionscrashes-aa77ca045e2e.json"

In [ ]:
from google.cloud import storage
from datetime import datetime

In [ ]:
local_file_path = "/Users/patkaa/Documents/Assignment 1 - DW - Fall 2025/Motor_Vehicle_Collisions_-_Crashes_20251118.csv"
bucket_name = "mvcollisions_bucket1" 

In [ ]:
#Checking for duplicates
MVCollisions.duplicated().sum()

In [ ]:
#Checking if there are null values
print(MVCollisions.isnull().sum())

In [ ]:
#Converting null values to “0”s
MVC_num_val = ['NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED']
MVCollisions[MVC_num_val] = MVCollisions[MVC_num_val].fillna(0).astype(int)

#Update the null values
MVCollisions['BOROUGH'] = MVCollisions['BOROUGH'].fillna('NP')
MVCollisions['ZIP CODE'] = MVCollisions['ZIP CODE'].fillna('NP')
MVCollisions['LATITUDE'] = MVCollisions['LATITUDE'].fillna('NP')
MVCollisions['LONGITUDE'] = MVCollisions['LONGITUDE'].fillna('NP')
MVCollisions['LOCATION'] = MVCollisions['LOCATION'].fillna('NP')
MVCollisions['ON STREET NAME'] = MVCollisions['ON STREET NAME'].fillna('NP')
MVCollisions['CROSS STREET NAME'] = MVCollisions['CROSS STREET NAME'].fillna('NP')
MVCollisions['OFF STREET NAME'] = MVCollisions['OFF STREET NAME'].fillna('NP')

MVCollisions['CONTRIBUTING FACTOR VEHICLE 1'] = MVCollisions['CONTRIBUTING FACTOR VEHICLE 1'].fillna('NA')
MVCollisions['CONTRIBUTING FACTOR VEHICLE 2'] = MVCollisions['CONTRIBUTING FACTOR VEHICLE 2'].fillna('NA')
MVCollisions['CONTRIBUTING FACTOR VEHICLE 3'] = MVCollisions['CONTRIBUTING FACTOR VEHICLE 3'].fillna('NA')
MVCollisions['CONTRIBUTING FACTOR VEHICLE 4'] = MVCollisions['CONTRIBUTING FACTOR VEHICLE 4'].fillna('NA')
MVCollisions['CONTRIBUTING FACTOR VEHICLE 5'] = MVCollisions['CONTRIBUTING FACTOR VEHICLE 5'].fillna('NA')
MVCollisions['VEHICLE TYPE CODE 1'] = MVCollisions['VEHICLE TYPE CODE 1'].fillna('NA')
MVCollisions['VEHICLE TYPE CODE 2'] = MVCollisions['VEHICLE TYPE CODE 2'].fillna('NA')
MVCollisions['VEHICLE TYPE CODE 3'] = MVCollisions['VEHICLE TYPE CODE 3'].fillna('NA')
MVCollisions['VEHICLE TYPE CODE 4'] = MVCollisions['VEHICLE TYPE CODE 4'].fillna('NA')
MVCollisions['VEHICLE TYPE CODE 5'] = MVCollisions['VEHICLE TYPE CODE 5'].fillna('NA')

In [ ]:
#Check if are Null values are removed
print(MVCollisions.isnull().sum())

In [ ]:
MVCollisions.head(20)

In [ ]:
#creating one date_time object for next steps
MVCollisions['CRASH_DATE_TIME'] = pd.to_datetime(
    MVCollisions['CRASH DATE'] + ' ' + MVCollisions['CRASH TIME'],
    errors='coerce'
)

In [ ]:
# Unified date format and Splitting the date into multiple unit 
MVCollisions['CRASH DATE_KEY'] = MVCollisions['CRASH_DATE_TIME'].dt.date
MVCollisions['CRASH YEAR'] = MVCollisions['CRASH_DATE_TIME'].dt.year
MVCollisions['CRASH QUARTER'] = MVCollisions['CRASH_DATE_TIME'].dt.quarter
MVCollisions['CRASH MONTH'] = MVCollisions['CRASH_DATE_TIME'].dt.month
MVCollisions['CRASH_DAY'] = MVCollisions['CRASH_DATE_TIME'].dt.day
MVCollisions['CRASH WEEK_DAY'] = MVCollisions['CRASH_DATE_TIME'].dt.day_name()
MVCollisions['CRASH HOUR'] = MVCollisions['CRASH_DATE_TIME'].dt.hour

In [ ]:
MVCollisions.head()

In [ ]:
# Summing two columns
MVCollisions['TOTAL_VICTIMS'] = MVCollisions['NUMBER OF PERSONS KILLED'] + MVCollisions['NUMBER OF PERSONS INJURED']

# Adding another new column - check if person was killed or surviwed
MVCollisions.loc[MVCollisions['NUMBER OF PERSONS KILLED'] > 0, 'IF_MORTAL'] = 1
MVCollisions.loc[MVCollisions['NUMBER OF PERSONS KILLED'] == 0, 'IF_MORTAL'] = 0

In [ ]:
print(MVCollisions.columns)

In [ ]:
MVCollisions.head()

In [ ]:
#Use the correct data type each new fact generated
MVCollisions['TOTAL_VICTIMS'] = MVCollisions['TOTAL_VICTIMS'].astype('int32') 
MVCollisions['IF_MORTAL'] = MVCollisions['IF_MORTAL'].astype('int')

In [ ]:
# Print final dataset and gain info
MVCollisions.head(100)

In [ ]:
print(MVCollisions)

In [ ]:
print(MVCollisions.info())

In [ ]:
#Building dimensional model
fact_collision = pd.DataFrame({
    "collision_key": pd.Series(dtype="int64"),           
    "collision_id": pd.Series(dtype="string"),
    "location_key": pd.Series(dtype="string"),
    "date_key": pd.Series(dtype="datetime64[ns]"),
    "time_key": pd.Series(dtype="datetime64[ns]"),
    "factor_key_1": pd.Series(dtype="string"),
    "factor_key_2": pd.Series(dtype="string"),
    "vehicle_key_1": pd.Series(dtype="string"),
    "vehicle_key_2": pd.Series(dtype="string"),
    "number_of_persons_injured": pd.Series(dtype="Int64"),
    "number_of_persons_killed": pd.Series(dtype="Int64"),
    "number_of_pedestrians_injured": pd.Series(dtype="Int64"),
    "number_of_pedestrians_killed": pd.Series(dtype="Int64"),
    "number_of_cyclist_injured": pd.Series(dtype="Int64"),
    "number_of_cyclist_killed": pd.Series(dtype="Int64"),
    "number_of_motorist_injured": pd.Series(dtype="Int64"),
    "number_of_motorist_killed": pd.Series(dtype="Int64")
})

fact_collision['collision_key'] = np.arange(1, len(fact_collision)+1)


dim_location = pd.DataFrame({
    "location_key": pd.Series(dtype="int64"),
    "location": pd.Series(dtype="string"),
    "latitude": pd.Series(dtype="string"),
    "longitude": pd.Series(dtype="string"),
    "borough": pd.Series(dtype="string"),
    "zip_code": pd.Series(dtype="string"),
    "on_street_name": pd.Series(dtype="string"),
    "off_street_name": pd.Series(dtype="string"),
    "cross_street_name": pd.Series(dtype="string")
})

dim_location['location_key'] = np.arange(1, len(dim_location)+1)


dim_date = pd.DataFrame({
    "date_key": pd.Series(dtype="int64"),   
    "crash_date": pd.Series(dtype="datetime64[ns]")
})

dim_date['date_key'] = np.arange(1, len(dim_date)+1)


dim_time = pd.DataFrame({
    "time_key": pd.Series(dtype="int64"),
    "crash_time": pd.Series(dtype="datetime64[ns]")
})

dim_time['time_key'] = np.arange(1, len(dim_time)+1)


dim_factor = pd.DataFrame({
    "factor_key_1": pd.Series(dtype="int64"),       
    "factor_key_2": pd.Series(dtype="string"),
    "contributing_factor_vehicle_1": pd.Series(dtype="string"),
    "contributing_factor_vehicle_2": pd.Series(dtype="string"),
    "contributing_factor_vehicle_3": pd.Series(dtype="string"),
    "contributing_factor_vehicle_4": pd.Series(dtype="string"),
    "contributing_factor_vehicle_5": pd.Series(dtype="string")
})

dim_factor['factor_key_1'] = np.arange(1, len(dim_factor)+1)


dim_vehicle = pd.DataFrame({
    "vehicle_key_1": pd.Series(dtype="int64"),    
    "vehicle_key_2": pd.Series(dtype="string"),
    "vehicle_type_code1": pd.Series(dtype="string"),
    "vehicle_type_code2": pd.Series(dtype="string"),
    "vehicle_type_code3": pd.Series(dtype="string"),
    "vehicle_type_code4": pd.Series(dtype="string"),
    "vehicle_type_code5": pd.Series(dtype="string")
})

dim_vehicle['vehicle_key_1'] = np.arange(1, len(dim_vehicle)+1)

In [ ]:
MVCollisions.head(10)

In [ ]:
MVCollisions.to_csv("MVCollisions.csv", index=False)